In [78]:
#importing dependencies
import matplotlib.pyplot as plt;
import pandas as pd;
import scipy.stats as st;

In [79]:
#Data files
mouse_metadata_path = 'Resources/Mouse_metadata.csv';
study_results_path = 'Resources/Study_results.csv';

#Reading in the data
mouse_metadata = pd.read_csv(mouse_metadata_path);
study_results = pd.read_csv(study_results_path);

#previewing column values for both data sets
print(mouse_metadata.columns);
print(study_results.columns);

#Merging the two data sets on their common column: Mouse ID
SCC_growth = mouse_metadata.merge(study_results , on= 'Mouse ID');

#Previewing merged data frame SCC_growth
SCC_growth.head(5)


Index(['Mouse ID', 'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)'], dtype='object')
Index(['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites'], dtype='object')


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [80]:
#Checking for mouse ID's with Duplicate time points
SCC_growth[['Mouse ID' , 'Timepoint']].value_counts()

SCC_growth = SCC_growth.drop_duplicates(subset=['Mouse ID' , 'Timepoint'] , keep=False )

SCC_growth[SCC_growth['Mouse ID'] == 'k403']
#try g989

Mouse ID  Timepoint
g989      20           2
          15           2
          10           2
          5            2
          0            2
                      ..
i334      20           1
          15           1
          10           1
          5            1
z969      45           1
Length: 1888, dtype: int64